In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install ultralytics==8.3.40
!pip install roboflow
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]

from detectron2.utils.logger import setup_logger
setup_logger()

from google.colab import drive
drive.mount('/content/gdrive')
ROOT_DIR = '/content/gdrive/My Drive/Georgios_Karagiannis/Code'

from ultralytics import YOLO
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.data import build_detection_test_loader
from roboflow import Roboflow

# Faster RCNN Evaluation for Level 1

In [ ]:
from detectron2.structures import Boxes
from torchvision.ops import box_iou
import torch
import numpy as np
import json
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

# Ρύθμιση Roboflow και Dataset
rf = Roboflow(api_key="jrkIR8tZdg5Y8uIaKmwZ")
project = rf.workspace("smart-fridge-jvm9v").project("level1_images")
version = project.version(2)
dataset = version.download("coco")

register_coco_instances("my_dataset_test", {}, "/content/level1_images-2/test/_annotations.coco.json", "/content/level1_images-2/test")

# Detectron2 Config
cfg = get_cfg()
cfg.merge_from_file("/content/gdrive/MyDrive/Georgios_Karagiannis/Code/config1.yml")
cfg.MODEL.WEIGHTS = "/content/gdrive/MyDrive/Georgios_Karagiannis/Code/D-level1/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

# Φόρτωση κατηγοριών
with open("/content/level1_images-2/test/_annotations.coco.json", "r") as f:
    coco_data = json.load(f)

categories = {cat["id"]: cat["name"] for cat in coco_data["categories"]}
target_category_names = ["Apple", "Banana", "Lime", "Orange", "Peach", "Pear", "Pomegranate", "Tomato"]
target_category_ids = [id for id, name in categories.items() if name in target_category_names]

val_loader = build_detection_test_loader(cfg, "my_dataset_test")  # Χρήση του loader

# 3. Αξιολόγηση με COCO Evaluator (Καλύτερο για Object Detection)
evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir="./output/")
coco_metrics = inference_on_dataset(predictor.model, val_loader, evaluator)
print("COCO Metrics:")
print(json.dumps(coco_metrics, indent=4))


# IoU threshold για επιτυχές ταίριασμα
iou_threshold = 0.85
y_true_cls = []
y_pred_cls = []

for d in DatasetCatalog.get("my_dataset_test"):
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)

    gt_boxes = [ann["bbox"] for ann in d["annotations"] if ann["category_id"] in target_category_ids]
    gt_classes = [ann["category_id"] for ann in d["annotations"] if ann["category_id"] in target_category_ids]

    if len(gt_boxes) == 0:
        continue

    gt_boxes_tensor = torch.tensor(gt_boxes, dtype=torch.float32)
    gt_boxes_tensor[:, 2] += gt_boxes_tensor[:, 0]
    gt_boxes_tensor[:, 3] += gt_boxes_tensor[:, 1]

    if "instances" in outputs:
        instances = outputs["instances"].to("cpu")
        pred_boxes_tensor = instances.pred_boxes.tensor
        pred_classes = instances.pred_classes.tolist()
    else:
        pred_boxes_tensor = torch.empty((0, 4))
        pred_classes = []

    matched_gt = set()
    matched_pred = set()

    if len(gt_boxes_tensor) > 0 and len(pred_boxes_tensor) > 0:
        ious = box_iou(gt_boxes_tensor, pred_boxes_tensor)
        for gt_idx in range(ious.shape[0]):
            pred_idx = torch.argmax(ious[gt_idx])
            if ious[gt_idx, pred_idx] >= iou_threshold:
                if gt_idx not in matched_gt and pred_idx.item() not in matched_pred:
                    matched_gt.add(gt_idx)
                    matched_pred.add(pred_idx.item())
                    y_true_cls.append(gt_classes[gt_idx])
                    y_pred_cls.append(pred_classes[pred_idx.item()])

    for gt_idx in range(len(gt_classes)):
        if gt_idx not in matched_gt:
            y_true_cls.append(gt_classes[gt_idx])
            y_pred_cls.append(-1)

    for pred_idx in range(len(pred_classes)):
        if pred_idx not in matched_pred:
            y_true_cls.append(-1)
            y_pred_cls.append(pred_classes[pred_idx])

# Ονόματα για αναφορά
all_labels = target_category_ids + [-1]
all_names = [categories[i] for i in target_category_ids] + ["Background"]

# Confusion Matrix
cm = confusion_matrix(y_true_cls, y_pred_cls, labels=all_labels)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=all_names,
            yticklabels=all_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title(f"Confusion Matrix")
plt.show()

# ➤ Classification Report
print("\n Classification Report (per class):")
report = classification_report(
    y_true_cls,
    y_pred_cls,
    labels=target_category_ids,
    target_names=target_category_names,
    zero_division=0,
    digits=4  # ➤ Εμφάνιση αποτελεσμάτων με 4 δεκαδικά ψηφία
)
print(report)

In [ ]:
from detectron2.structures import Boxes
from torchvision.ops import box_iou
import torch
import numpy as np
import json
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns
import cv2


# Ρύθμιση Roboflow και Dataset
rf = Roboflow(api_key="jrkIR8tZdg5Y8uIaKmwZ")
project = rf.workspace("smart-fridge-jvm9v").project("level1_images")
version = project.version(2)
dataset = version.download("coco")

# Κάνουμε Register Test και Validation Sets
register_coco_instances("my_dataset_test", {}, "/content/level1_images-2/test/_annotations.coco.json", "/content/level1_images-2/test")
register_coco_instances("my_dataset_val", {}, "/content/level1_images-2/valid/_annotations.coco.json", "/content/level1_images-2/valid")

# Φόρτωση Κατηγοριών
with open("/content/level1_images-2/test/_annotations.coco.json", "r") as f:
    coco_data = json.load(f)

categories = {cat["id"]: cat["name"] for cat in coco_data["categories"]}
target_category_names = ["Apple", "Banana", "Lime", "Orange", "Peach", "Pear", "Pomegranate", "Tomato"]
target_category_ids = [id for id, name in categories.items() if name in target_category_names]

# Detectron2 Config
cfg = get_cfg()
cfg.merge_from_file("/content/gdrive/MyDrive/Georgios_Karagiannis/Code/config1.yml")
cfg.MODEL.WEIGHTS = "/content/gdrive/MyDrive/Georgios_Karagiannis/Code/D-level1/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0
predictor = DefaultPredictor(cfg)

# Φόρτωση Test και Validation Set μαζί
test_dataset = DatasetCatalog.get("my_dataset_test")
val_dataset = DatasetCatalog.get("my_dataset_val")
combined_dataset = test_dataset + val_dataset

# Καμπύλες αξιολόγησης
iou_threshold = 0.5
y_true_all = {cat_id: [] for cat_id in target_category_ids}
y_scores_all = {cat_id: [] for cat_id in target_category_ids}

# Αξιολόγηση
for d in combined_dataset:
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)

    gt_boxes = [ann["bbox"] for ann in d["annotations"] if ann["category_id"] in target_category_ids]
    gt_classes = [ann["category_id"] for ann in d["annotations"] if ann["category_id"] in target_category_ids]

    if len(gt_boxes) == 0:
        continue

    gt_boxes_tensor = torch.tensor(gt_boxes, dtype=torch.float32)
    gt_boxes_tensor[:, 2] += gt_boxes_tensor[:, 0]
    gt_boxes_tensor[:, 3] += gt_boxes_tensor[:, 1]

    if "instances" in outputs:
        instances = outputs["instances"].to("cpu")
        pred_boxes_tensor = instances.pred_boxes.tensor
        pred_classes = instances.pred_classes.tolist()
        scores = instances.scores.tolist()
    else:
        pred_boxes_tensor = torch.empty((0, 4))
        pred_classes = []
        scores = []

    matched_gt = set()
    matched_pred = set()

    if len(gt_boxes_tensor) > 0 and len(pred_boxes_tensor) > 0:
        ious = box_iou(gt_boxes_tensor, pred_boxes_tensor)
        for gt_idx in range(ious.shape[0]):
            pred_idx = torch.argmax(ious[gt_idx])
            if ious[gt_idx, pred_idx] >= iou_threshold:
                if gt_idx not in matched_gt and pred_idx.item() not in matched_pred:
                    matched_gt.add(gt_idx)
                    matched_pred.add(pred_idx.item())
                    gt_label = gt_classes[gt_idx]
                    pred_label = pred_classes[pred_idx.item()]
                    score = scores[pred_idx.item()]
                    for cat_id in target_category_ids:
                        if pred_label == cat_id:
                            y_true_all[cat_id].append(int(gt_label == cat_id))
                            y_scores_all[cat_id].append(score)

# Δημιουργία Καμπυλών με εμφάνιση όπως του YOLO
plt.figure(figsize=(18, 12))

# 1. Precision-Recall Curve
plt.subplot(2, 2, 1)
for cat_id in target_category_ids:
    if len(y_true_all[cat_id]) > 0:
        precision, recall, _ = precision_recall_curve(y_true_all[cat_id], y_scores_all[cat_id])
        plt.plot(recall, precision, label=categories[cat_id])
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()

# 2. Precision-Confidence Curve (βελτιωμένη εμφάνιση)
plt.subplot(2, 2, 2)
all_precisions = []
all_thresholds = []
for cat_id in target_category_ids:
    if len(y_true_all[cat_id]) > 0:
        precision, recall, thresholds = precision_recall_curve(y_true_all[cat_id], y_scores_all[cat_id])
        all_precisions.append(precision[:-1])
        all_thresholds.append(thresholds)
        plt.plot(thresholds, precision[:-1], label=categories[cat_id])
min_len = min(len(p) for p in all_precisions)
avg_precision = np.mean([p[:min_len] for p in all_precisions], axis=0)
avg_threshold = all_thresholds[0][:min_len]
plt.plot(avg_threshold, avg_precision, label="all classes", linewidth=3, color="blue")
plt.xlabel("Confidence")
plt.ylabel("Precision")
plt.title("Precision-Confidence Curve")
plt.legend()

# 3. Recall-Confidence Curve
plt.subplot(2, 2, 3)
all_recalls = []
all_thresholds = []
for cat_id in target_category_ids:
    if len(y_true_all[cat_id]) > 0:
        precision, recall, thresholds = precision_recall_curve(y_true_all[cat_id], y_scores_all[cat_id])
        all_recalls.append(recall[:-1])
        all_thresholds.append(thresholds)
        plt.plot(thresholds, recall[:-1], label=categories[cat_id])
min_len = min(len(p) for p in all_recalls)
avg_recall = np.mean([p[:min_len] for p in all_recalls], axis=0)
avg_threshold = all_thresholds[0][:min_len]
plt.plot(avg_threshold, avg_recall, label="all classes", linewidth=3, color="blue")
plt.xlabel("Confidence")
plt.ylabel("Recall")
plt.title("Recall-Confidence Curve")
plt.legend()

# 4. F1-Confidence Curve
plt.subplot(2, 2, 4)
all_f1 = []
all_thresholds = []
for cat_id in target_category_ids:
    if len(y_true_all[cat_id]) > 0:
        precision, recall, thresholds = precision_recall_curve(y_true_all[cat_id], y_scores_all[cat_id])
        f1_scores = [2 * p * r / (p + r + 1e-8) for p, r in zip(precision[:-1], recall[:-1])]
        all_f1.append(f1_scores)
        all_thresholds.append(thresholds)
        plt.plot(thresholds, f1_scores, label=categories[cat_id])
min_len = min(len(p) for p in all_f1)
avg_f1 = np.mean([p[:min_len] for p in all_f1], axis=0)
avg_threshold = all_thresholds[0][:min_len]
plt.plot(avg_threshold, avg_f1, label="all classes", linewidth=3, color="blue")
plt.xlabel("Confidence")
plt.ylabel("F1 Score")
plt.title("F1-Confidence Curve")
plt.legend()

plt.tight_layout()
plt.show()


# Faster RCNN Evaluation for Level 2

In [ ]:
from detectron2.structures import Boxes
from torchvision.ops import box_iou
import torch
import numpy as np
import json
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

rf = Roboflow(api_key="jrkIR8tZdg5Y8uIaKmwZ")
project = rf.workspace("smart-fridge-jvm9v").project("smart-fridge-p5rw0")
version = project.version(9)
dataset = version.download("coco")

register_coco_instances("test", {}, "/content/Smart-Fridge-9/test/_annotations.coco.json", "/content/Smart-Fridge-9/test")

# 1. Ρύθμιση Detectron2 και Μοντέλου
cfg = get_cfg()
cfg.merge_from_file("/content/gdrive/MyDrive/Georgios_Karagiannis/Code/config.yml")  # Διόρθωσε την διαδρομή αν χρειάζεται
cfg.MODEL.WEIGHTS = "/content/gdrive/MyDrive/Georgios_Karagiannis/Code/D-level2/model_final.pth" # Διόρθωσε την διαδρομή
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0  # Αυξημένο threshold για πιο ακριβείς μετρήσεις

predictor = DefaultPredictor(cfg)

# 2. Φόρτωση και Προετοιμασία Δεδομένων
json_path = "/content/Smart-Fridge-9/test/_annotations.coco.json"  # Διόρθωσε την διαδρομή
with open(json_path, "r") as f:
    coco_data = json.load(f)

# ΠΡΟΣΟΧΗ: ΤΩΡΑ ΤΟ coco_data ΕΙΝΑΙ ΛΙΣΤΑ!  Πρέπει να πάρεις το πρώτο στοιχείο.
#coco_data = coco_data[0]  # Πάρε το πρώτο (και μοναδικό) λεξικό μέσα στη λίστα

# Δημιουργία αντιστοίχισης ονομάτων κατηγοριών
categories = {cat["id"]: cat["name"] for cat in coco_data["categories"]}
target_category_names = ["half-ripe", "over-ripe", "ripe", "unripe"]
target_category_ids = [id for id, name in categories.items() if name in target_category_names]
print(f"Target category IDs: {target_category_ids}")

val_loader = build_detection_test_loader(cfg, "test")  # Χρήση του loader

# 3. Αξιολόγηση με COCO Evaluator (Καλύτερο για Object Detection)
evaluator = COCOEvaluator("test", cfg, False, output_dir="./output/")
coco_metrics = inference_on_dataset(predictor.model, val_loader, evaluator)
print("COCO Metrics:")
print(json.dumps(coco_metrics, indent=4))

iou_threshold = 0.0
y_true_cls = []
y_pred_cls = []

for d in DatasetCatalog.get("test"):
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)

    gt_boxes = [ann["bbox"] for ann in d["annotations"] if ann["category_id"] in target_category_ids]
    gt_classes = [ann["category_id"] for ann in d["annotations"] if ann["category_id"] in target_category_ids]

    if len(gt_boxes) == 0:
        continue

    gt_boxes_tensor = torch.tensor(gt_boxes, dtype=torch.float32)
    gt_boxes_tensor[:, 2] += gt_boxes_tensor[:, 0]
    gt_boxes_tensor[:, 3] += gt_boxes_tensor[:, 1]

    if "instances" in outputs:
        instances = outputs["instances"].to("cpu")
        pred_boxes_tensor = instances.pred_boxes.tensor
        pred_classes = instances.pred_classes.tolist()
    else:
        pred_boxes_tensor = torch.empty((0, 4))
        pred_classes = []

    matched_gt = set()
    matched_pred = set()

    if len(gt_boxes_tensor) > 0 and len(pred_boxes_tensor) > 0:
        ious = box_iou(gt_boxes_tensor, pred_boxes_tensor)
        for gt_idx in range(ious.shape[0]):
            pred_idx = torch.argmax(ious[gt_idx])
            if ious[gt_idx, pred_idx] >= iou_threshold:
                if gt_idx not in matched_gt and pred_idx.item() not in matched_pred:
                    matched_gt.add(gt_idx)
                    matched_pred.add(pred_idx.item())
                    y_true_cls.append(gt_classes[gt_idx])
                    y_pred_cls.append(pred_classes[pred_idx.item()])

    # False Negatives
    for gt_idx in range(len(gt_classes)):
        if gt_idx not in matched_gt:
            y_true_cls.append(gt_classes[gt_idx])
            y_pred_cls.append(-1)

    # False Positives
    for pred_idx in range(len(pred_classes)):
        if pred_idx not in matched_pred:
            y_true_cls.append(-1)
            y_pred_cls.append(pred_classes[pred_idx])

# Ονόματα για αναφορά
all_labels = target_category_ids + [-1]
all_names = [categories[i] for i in target_category_ids] + ["Background"]

# Confusion Matrix
cm = confusion_matrix(y_true_cls, y_pred_cls, labels=all_labels)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=all_names,
            yticklabels=all_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title(f"Confusion Matrix")
plt.show()
# Classification Report
print("\nClassification Report (per class):")
report = classification_report(
    y_true_cls,
    y_pred_cls,
    labels=target_category_ids,
    target_names=[categories[i] for i in target_category_ids],
    zero_division=0,
    digits=4
)
print(report)

In [ ]:
from detectron2.structures import Boxes
from torchvision.ops import box_iou
import torch
import numpy as np
import json
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns
import cv2


# Κάνουμε Register Test και Validation Sets
register_coco_instances("my_dataset_test", {}, "/content/Smart-Fridge-9/test/_annotations.coco.json", "/content/Smart-Fridge-9/test")
register_coco_instances("my_dataset_val", {}, "/content/Smart-Fridge-9/valid/_annotations.coco.json", "/content/Smart-Fridge-9/valid")

# 2. Φόρτωση και Προετοιμασία Δεδομένων
json_path = "/content/Smart-Fridge-9/test/_annotations.coco.json"  # Διόρθωσε την διαδρομή
with open(json_path, "r") as f:
    coco_data = json.load(f)

# ΠΡΟΣΟΧΗ: ΤΩΡΑ ΤΟ coco_data ΕΙΝΑΙ ΛΙΣΤΑ!  Πρέπει να πάρεις το πρώτο στοιχείο.
#coco_data = coco_data[0]  # Πάρε το πρώτο (και μοναδικό) λεξικό μέσα στη λίστα

# Δημιουργία αντιστοίχισης ονομάτων κατηγοριών
categories = {cat["id"]: cat["name"] for cat in coco_data["categories"]}
target_category_names = ["half-ripe", "over-ripe", "ripe", "unripe"]
target_category_ids = [id for id, name in categories.items() if name in target_category_names]
print(f"Target category IDs: {target_category_ids}")


# Φόρτωση Test και Validation Set μαζί
test_dataset = DatasetCatalog.get("my_dataset_test")
val_dataset = DatasetCatalog.get("my_dataset_val")
combined_dataset = test_dataset + val_dataset

# Καμπύλες αξιολόγησης
iou_threshold = 0.5
y_true_all = {cat_id: [] for cat_id in target_category_ids}
y_scores_all = {cat_id: [] for cat_id in target_category_ids}

# Αξιολόγηση
for d in combined_dataset:
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)

    gt_boxes = [ann["bbox"] for ann in d["annotations"] if ann["category_id"] in target_category_ids]
    gt_classes = [ann["category_id"] for ann in d["annotations"] if ann["category_id"] in target_category_ids]

    if len(gt_boxes) == 0:
        continue

    gt_boxes_tensor = torch.tensor(gt_boxes, dtype=torch.float32)
    gt_boxes_tensor[:, 2] += gt_boxes_tensor[:, 0]
    gt_boxes_tensor[:, 3] += gt_boxes_tensor[:, 1]

    if "instances" in outputs:
        instances = outputs["instances"].to("cpu")
        pred_boxes_tensor = instances.pred_boxes.tensor
        pred_classes = instances.pred_classes.tolist()
        scores = instances.scores.tolist()
    else:
        pred_boxes_tensor = torch.empty((0, 4))
        pred_classes = []
        scores = []

    matched_gt = set()
    matched_pred = set()

    if len(gt_boxes_tensor) > 0 and len(pred_boxes_tensor) > 0:
        ious = box_iou(gt_boxes_tensor, pred_boxes_tensor)
        for gt_idx in range(ious.shape[0]):
            pred_idx = torch.argmax(ious[gt_idx])
            if ious[gt_idx, pred_idx] >= iou_threshold:
                if gt_idx not in matched_gt and pred_idx.item() not in matched_pred:
                    matched_gt.add(gt_idx)
                    matched_pred.add(pred_idx.item())
                    gt_label = gt_classes[gt_idx]
                    pred_label = pred_classes[pred_idx.item()]
                    score = scores[pred_idx.item()]
                    for cat_id in target_category_ids:
                        if pred_label == cat_id:
                            y_true_all[cat_id].append(int(gt_label == cat_id))
                            y_scores_all[cat_id].append(score)

# Δημιουργία Καμπυλών με εμφάνιση όπως του YOLO
plt.figure(figsize=(18, 12))

# 1. Precision-Recall Curve
plt.subplot(2, 2, 1)
for cat_id in target_category_ids:
    if len(y_true_all[cat_id]) > 0:
        precision, recall, _ = precision_recall_curve(y_true_all[cat_id], y_scores_all[cat_id])
        plt.plot(recall, precision, label=categories[cat_id])
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()

# 2. Precision-Confidence Curve (βελτιωμένη εμφάνιση)
plt.subplot(2, 2, 2)
all_precisions = []
all_thresholds = []
for cat_id in target_category_ids:
    if len(y_true_all[cat_id]) > 0:
        precision, recall, thresholds = precision_recall_curve(y_true_all[cat_id], y_scores_all[cat_id])
        all_precisions.append(precision[:-1])
        all_thresholds.append(thresholds)
        plt.plot(thresholds, precision[:-1], label=categories[cat_id])
min_len = min(len(p) for p in all_precisions)
avg_precision = np.mean([p[:min_len] for p in all_precisions], axis=0)
avg_threshold = all_thresholds[0][:min_len]
plt.plot(avg_threshold, avg_precision, label="all classes", linewidth=3, color="blue")
plt.xlabel("Confidence")
plt.ylabel("Precision")
plt.title("Precision-Confidence Curve")
plt.legend()

# 3. Recall-Confidence Curve
plt.subplot(2, 2, 3)
all_recalls = []
all_thresholds = []
for cat_id in target_category_ids:
    if len(y_true_all[cat_id]) > 0:
        precision, recall, thresholds = precision_recall_curve(y_true_all[cat_id], y_scores_all[cat_id])
        all_recalls.append(recall[:-1])
        all_thresholds.append(thresholds)
        plt.plot(thresholds, recall[:-1], label=categories[cat_id])
min_len = min(len(p) for p in all_recalls)
avg_recall = np.mean([p[:min_len] for p in all_recalls], axis=0)
avg_threshold = all_thresholds[0][:min_len]
plt.plot(avg_threshold, avg_recall, label="all classes", linewidth=3, color="blue")
plt.xlabel("Confidence")
plt.ylabel("Recall")
plt.title("Recall-Confidence Curve")
plt.legend()

# 4. F1-Confidence Curve
plt.subplot(2, 2, 4)
all_f1 = []
all_thresholds = []
for cat_id in target_category_ids:
    if len(y_true_all[cat_id]) > 0:
        precision, recall, thresholds = precision_recall_curve(y_true_all[cat_id], y_scores_all[cat_id])
        f1_scores = [2 * p * r / (p + r + 1e-8) for p, r in zip(precision[:-1], recall[:-1])]
        all_f1.append(f1_scores)
        all_thresholds.append(thresholds)
        plt.plot(thresholds, f1_scores, label=categories[cat_id])
min_len = min(len(p) for p in all_f1)
avg_f1 = np.mean([p[:min_len] for p in all_f1], axis=0)
avg_threshold = all_thresholds[0][:min_len]
plt.plot(avg_threshold, avg_f1, label="all classes", linewidth=3, color="blue")
plt.xlabel("Confidence")
plt.ylabel("F1 Score")
plt.title("F1-Confidence Curve")
plt.legend()

plt.tight_layout()
plt.show()
